# **Measure Organelle Morphology**

***Prior to this notebook, you should have already run through [2.0_quantification_setup](2.0_quantification_setup.ipynb).***

In notebooks 2.1 through 2.4, we will go over the implementation of `infer-subc` quantification methods (explained in detail in the `method_...` notebooks) to assess the morphology, interactions, and distribution of organelles at the single-cell level. 

### 📍 **Purpose**

This notebook measures the amount, size, and shape of organelles. It includes an option to batch process the morphology measurements for `mulitple organelles` across `multiple cells`.

### 🍃 **Biological Relevance**
Measurements of organelle morphology -- amount, size, and shape -- are included as part of the organelle signature analysis. These metrics can provide information about the physiology of a cell and its constituent organelles. 

Organelle amounts have been demonstrated to differ between cell types. The cytoplasms of some specialized cell types, like adipocytes, are composed almost entirely by a single large lipid droplet for fat storage[[1]](https://doi.org/10.3390/biom11121906), while other cell types, like muscle cells, have large and elaborate mitochdondrial networks for effective metabolite diffusion during muscle contraction[[2]](https://doi.org/10.1038/nature14614).

Additionally, different organelle morphologies are important to their function. For example, fission and fusion maintain mitochondrial homeostasis by modulating the size and connectedness of the mitochondrial network. A recent study demonstrated that asymmetric fission events resulted in morphologically distinct daughter mitochondria with different fates; the larger mitochondria continued to grow and divide, while the smaller, spherical fragments containing high reactive oxygen species (ROS) were destined for autophagic degradation[[3]](https://doi.org/10.1038/s41586-021-03510-6).

### 📐 **Regionprops Morphology Measurements** 

*You can learn more about the implementation of regionprops within infer-subc in the [method_morphology](method_morphology.ipynb) notebook.*

The following morphological measurements are included:
- `label`: the unique ID number for the object being measured
- `centroid`: centroid coordinate tuple (row, col, Z)
- `bbox`: bounding box coordinates (min_row, min_col, max_row, max_col); pixels/voxels belonging to the bounding box are in the half-open interval [min_row; max_row) and [min_col; max_col).
- `area`: (or `volume` for 3D z-stack images) area of the region i.e. number of pixels of the region scaled by pixel-area; this metric has the option to be converted into "real world" units using the scale from the metadata.
- `equivalent_diameter`: the diameter of a circle with the same area as the region; this metric has the option to be converted into "real world" units using the scale from the metadata.
- `extent`: ratio of pixels/voxels in the region to pixels/voxels in the total bounding box. Computed as area / (rows * cols)
- `euler_number`: Euler characteristic of the set of non-zero pixels. Computed as number of connected components subtracted by number of holes (input.ndim connectivity). In 3D, number of connected components plus number of holes subtracted by number of tunnels.
- `solidity`: ratio of pixels/voxels in the region to pixels/voxels of the convex hull image.
- `axis_major_length`: the length of the major axis of the ellipse that has the same normalized second central moments as the region; this metric has the option to be converted into "real world" units using the scale from the metadata.
- `surface_area`: the surface area of the region. For 3D, surface area of a 2D surface mesh of the region (skimage.measure.marching_cubes) using skimage.measure.mesh_surface_area; this metric has the option to be converted into "real world" units using the scale from the metadata.
- `SA_to_volume`: surface area / area (or volume); this metric has the option to be converted into "real world" units using the scale from the metadata.

The following measures of the intensity images are also included:
- `min_intensity`: value with the least intensity in the region.
- `max_intensity`: value with the greatest intensity in the region.
- `mean_intensity`: value with the mean intensity in the region.
- `standard_deviation_intensity`: the standard deviation of the intensity in the region.

These measurements and definitions are derived from the [`skimage.measure.regionprops()`](https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops) function. More in depth information about each measurement can be found there.

---------------------
## **IMPORTS AND LOAD IMAGE**
Details about the functions included in this subsection are outlined in the [`2.0_quantification_setup`](2.0_quantification_setup.ipynb) notebook. Please visit that notebook first if you are confused about any of the code included here.

In [1]:
from typing import List, Union
from pathlib import Path
import os
import time
import warnings

from infer_subc.core.img import *

import numpy as np
import pandas as pd
import napari
from napari.utils.notebook_display import nbscreenshot

from infer_subc.utils.stats import get_morphology_metrics
from infer_subc.utils.batch import list_image_files, find_segmentation_tiff_files
from infer_subc.core.file_io import read_czi_image, read_tiff_image

pd.set_option('display.max_columns', None)

#### &#x1F3C3; **Run code; no user input required**

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following information about your data: `raw_img_type`, `data_root_path`, `raw_data_path`, `seg_data_path`, and `quant_data_path`.

In [2]:
#### USER INPUT REQUIRED ###
raw_img_type = ".czi"
data_root_path = Path(os.path.expanduser("~")) / "Documents/Python_Scripts/Infer-subc"
raw_data_path = data_root_path / "raw_two"
seg_data_path = data_root_path / "out_two"
quant_data_path = data_root_path / "quant_two"

#### &#x1F3C3; **Run code; no user input required**

In [3]:
# Create the output directory to save the segmentation outputs in.
if not Path.exists(quant_data_path):
    Path.mkdir(quant_data_path)
    print(f"making {quant_data_path}")

# Create a list of the file paths for each image in the input folder. Select test image path.
raw_img_file_list = list_image_files(raw_data_path,raw_img_type)
pd.set_option('display.max_colwidth', None)
pd.DataFrame({"Image Name":raw_img_file_list})

,Image Name
0,C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw_two\a24hrs-Ctrl_14_Unmixing.czi
1,C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw_two\a48hrs-Ctrl + oleic acid_01_Unmixing.czi


#### &#x1F6D1; &#x270D; **User Input Required:**

Use the list above to specify which image you wish to analyze based on its index: `test_img_n`

In [4]:
#### USER INPUT REQUIRED ###
test_img_n = 0

#### &#x1F3C3; **Run code; no user input required**

In [5]:
# Read in the image and metadata as an ndarray and dictionary from the test image selected above. 
test_img_name = raw_img_file_list[test_img_n]
img_data,meta_dict = read_czi_image(test_img_name)

# Define some of the metadata features.
channel_names = meta_dict['name']
meta = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']
file_path = meta_dict['file_name']

print("Metadata information")
print(f"File path: {file_path}")
for i in list(range(len(channel_names))):
    print(f"Channel {i} name: {channel_names[i]}")
print(f"Scale (ZYX): {scale}")
print(f"Channel axis: {channel_axis}")

Metadata information
File path: C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw_two\a24hrs-Ctrl_14_Unmixing.czi
Channel 0 name: 0 :: a24hrs-Ctrl_14_Unmixing-0 :: Nuclei_Jan22
Channel 1 name: 0 :: a24hrs-Ctrl_14_Unmixing-0 :: Lyso+405_Jan22
Channel 2 name: 0 :: a24hrs-Ctrl_14_Unmixing-0 :: Mito+405_Jan22
Channel 3 name: 0 :: a24hrs-Ctrl_14_Unmixing-0 :: Golgi+405_Jan22
Channel 4 name: 0 :: a24hrs-Ctrl_14_Unmixing-0 :: Peroxy+405_Jan22
Channel 5 name: 0 :: a24hrs-Ctrl_14_Unmixing-0 :: ER+405_Jan22
Channel 6 name: 0 :: a24hrs-Ctrl_14_Unmixing-0 :: BODIPY+405low_Jan22
Channel 7 name: 0 :: a24hrs-Ctrl_14_Unmixing-0 :: Residuals
Scale (ZYX): (0.3891184878080979, 0.07987165184837317, 0.07987165184837318)
Channel axis: 0


#### &#x1F6D1; &#x270D; **User Input Required:**

Specify the following information about the segmentation files: - `org_file_names`, `org_channels_ordered`, `regions_file_names`, `suffix_separator`, and `mask_name`.

In [6]:
#### USER INPUT REQUIRED ###
org_file_names = ["lyso", "mito", "golgi", "perox", "ER", "LD"]
org_channels_ordered = [1, 2, 3, 4, 5, 6]
regions_file_names = ["cell", "nuc"]
suffix_separator = "-20230426_test_"
mask_name = "cell"

#### &#x1F3C3; **Run code; no user input required**

In [7]:
# find file paths for segmentations
all_suffixes = org_file_names + regions_file_names
filez = find_segmentation_tiff_files(file_path, all_suffixes, seg_data_path, suffix_separator)

# read the segmentation and masks/regions files into memory
organelles = [read_tiff_image(filez[org]) for org in org_file_names]
regions = [] 
for m in regions_file_names:
    mfile = read_tiff_image(filez[m])
    regions.append(mfile)

# match the intensity channels to the segmentation files
intensities = [img_data[ch] for ch in org_channels_ordered]

# open viewer and add images
viewer = napari.Viewer()
for r, reg in enumerate(regions_file_names):
    viewer.add_image(regions[r],
                     scale=scale,
                     name=f"{reg} mask")

# colors = ["red", "bop orange", "yellow", "green", "blue", "cyan", "magenta", "bop purple"]
for o, org in enumerate(org_file_names):
    viewer.add_image(intensities[o],
                     scale=scale,
                     name=f"{org} intensity channel")
    viewer.add_labels(organelles[o],
                      scale=scale,
                      name=f"{org} segmentation")
viewer.grid.enabled = True
viewer.reset_view()

print("The following matching files were found and can now be viewed in Napari:")
filez

04-Dec-24 13:11:06 - vispy    - WARNING  - QWindowsWindow::setGeometry: Unable to set geometry 1086x657+1532+206 (frame: 1104x704+1523+168) on QWidgetWindow/"_QtMainWindowClassWindow" on "\\.\DISPLAY1". Resulting geometry: 905x825+1530+199 (frame: 923x872+1521+161) margins: 9, 38, 9, 9 minimum size: 385x515 MINMAXINFO maxSize=0,0 maxpos=0,0 mintrack=403,562 maxtrack=0,0)


The following matching files were found and can now be viewed in Napari:


{'raw': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/raw_two/a24hrs-Ctrl_14_Unmixing.czi'),
 'lyso': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_two/a24hrs-Ctrl_14_Unmixing-20230426_test_lyso.tiff'),
 'mito': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_two/a24hrs-Ctrl_14_Unmixing-20230426_test_mito.tiff'),
 'golgi': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_two/a24hrs-Ctrl_14_Unmixing-20230426_test_golgi.tiff'),
 'perox': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_two/a24hrs-Ctrl_14_Unmixing-20230426_test_perox.tiff'),
 'ER': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_two/a24hrs-Ctrl_14_Unmixing-20230426_test_ER.tiff'),
 'LD': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_two/a24hrs-Ctrl_14_Unmixing-20230426_test_LD.tiff'),
 'cell': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out_two/a24h

------

## **EXPLANATION OF STEPS**

### 🦠 **Processing One or More Organelles from <ins>ONE CELL</ins>**

#### **`STEP 1` - Select mask to be used**

##### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** To ensure we are performing single cell analysis, we will apply the cell segmentation as a mask to the segmentation file. This will exclude any objects outside of the mask area from the analysis. The mask file is selected from the list of regions and added to Napari for visual inspection if desirer.

In [ ]:
# select the mask from the region list
mask = regions[regions_file_names.index(mask_name)]

# add mask to napari for visual inspection
viewer.layers.clear()
viewer.add_image(img_data, scale=scale, name="Intensity Image")
viewer.add_labels(mask, scale=scale, name="Mask")
viewer.grid.enabled = False
viewer.reset_view()

#### **`STEP 2` - Loop through the list of organelles to quantify the morphology of each**

##### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** The block of code below loops through the list of organelles to:
1) Select the intensity image that organelle segmentation was derived from
2) Select the organelle segmentation image
3) Assure the segmentation files are formatted corrected (e.g., the ER should only include on object)

> ***IMPORTANT**: The solidity measurement may cause an error for objects that are very small. It depends on the convex hull measurement which is computed as 0 when the object volume is very small; the solidity value is them output in the table as `inf` (infinity). The following error message will be included:*
> ```python
> UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
> ```

In [ ]:
# empty list to collect a morphology data for each organelle
org_tabs = []

# loop through the list of organelles and run the get_morphology_metrics function
for j, target in enumerate(org_file_names):
    # select intensity image
    org_img = intensities[j]  
    
    # select segmentation and if ER, ensure it is only one object
    if target == 'ER':
        org_obj = (organelles[j] > 0).astype(np.uint16)  
    else:
        org_obj = organelles[j]
    
    # run get_morphology_metrics function to output a table of measurements
    org_metrics = get_morphology_metrics(segmentation_img=org_obj, 
                                        seg_name=target,
                                        intensity_img=org_img, 
                                        mask=mask,
                                        scale=scale)

    # add table to list above
    org_tabs.append(org_metrics)

# print each table separately
for i, org in enumerate(org_file_names):
    print(f"{org} morphology metrics table:")
    display(org_tabs[i])

#### **`STEP 3` - Combine all of the tables together and add column**

##### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block combines the above tables together so that each organelle object is listed as a separate column in a single table. A new column is then added to specify which image the data is from.

In [ ]:
# combine the lists for each organelle into one table
final_org_tab = pd.concat(org_tabs, ignore_index=True)

# add a new column to list the name of the image these data are derived from 
final_org_tab.insert(loc=0,column='image_name',value=file_path.stem)

# print table for inspection
display(final_org_tab)

#### **`DEFINE` - The get_organelle_morph() function**

> ***IMPORTANT**: The solidity measurement may cause an error for objects that are very small. It depends on the convex hull measurement which is computed as 0 when the object volume is very small; the solidity value is them output in the table as `inf` (infinity). The following error message will be included:*
> ```python
> UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
> ```

In [10]:
def _get_org_morphology(source_file: str,
                         list_obj_names: List[str],
                         list_obj_segs: List[np.ndarray],
                         list_intensity_img: List[np.ndarray],
                         list_region_names: List[str],
                         list_region_segs: List[np.ndarray],
                         mask: str,
                         scale: Union[tuple,None] = None):
    """
    Measure the amount, size, and shape of multiple organelles from a single cell

    Parameters:
    ----------
    source_file: str
        file path; this is used for recorder keeping of the file name in the output data tables
    list_obj_names: List[str]
        a list of object names (strings) that will be measured; this should match the order in list_obj_segs
    list_obj_segs: List[np.ndarray]
        a list of 3D (ZYX) segmentation np.ndarrays that will be measured per cell; the order should match the list_obj_names 
    list_intensity_img: List[np.ndarray]
        a list of 3D (ZYX) grayscale np.ndarrays that will be used to measure fluoresence intensity in each region and object
    list_region_names: List[str]
        a list of region names (strings); these should include the mask (entire region being measured - usually the cell) 
        and other sub-mask regions from which we can meausure the objects in (ex - nucleus, neurites, soma, etc.). It should 
        also include the centering object used when created the XY distribution bins.
        The order should match the list_region_segs
    list_region_segs: List[np.ndarray]
        a list of 3D (ZYX) binary np.ndarrays of the region masks; the order should match the list_region_names.
    mask: str
        a str of which region name (contained in the list_region_names list) should be used as the main mask (e.g., cell mask)
    scale: Union[tuple,None] = None
        a tuple that contains the real world dimensions for each dimension in the image (Z, Y, X)

    Returns:
    ----------
    Dataframe of measurements of organelle morphology

    """
    print(f"Quantifying organelle morphology from {source_file}.")

    # select the mask from the region list
    mask = list_region_segs[list_region_names.index(mask)]
    
    # empty list to collect a morphology data for each organelle
    org_tabs = []

    # loop through the list of organelles and run the get_morphology_metrics function
    for j, target in enumerate(list_obj_names):
        # select intensity image
        org_img = list_intensity_img[j]  
        
        # select segmentation and if ER, ensure it is only one object
        if target == 'ER':
            org_obj = (list_obj_segs[j] > 0).astype(np.uint16)  
        else:
            org_obj = list_obj_segs[j]
        
        # run get_morphology_metrics function to output a table of measurements
        org_metrics = get_morphology_metrics(segmentation_img=org_obj, 
                                            seg_name=target,
                                            intensity_img=org_img, 
                                            mask=mask,
                                            scale=scale)

        # add table to list above
        org_tabs.append(org_metrics)

    # combine the lists for each organelle into one table
    final_org_tab = pd.concat(org_tabs, ignore_index=True)

    # add a new column to list the name of the image these data are derived from 
    final_org_tab.insert(loc=0,column='image_name',value=source_file.stem)

    return final_org_tab

In [11]:
org_morph_tab = _get_org_morphology(source_file = file_path,
                                     list_obj_names = org_file_names,
                                     list_obj_segs = organelles,
                                     list_intensity_img = intensities, 
                                     list_region_names = regions_file_names,
                                     list_region_segs = regions,
                                     mask=mask_name,
                                     scale=scale)
org_morph_tab

Quantifying organelle morphology from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw_two\a24hrs-Ctrl_14_Unmixing.czi.
Warning(s) suppressed while quantifying lyso.
Warning(s) suppressed while quantifying mito.
Warning(s) suppressed while quantifying golgi.
Warning(s) suppressed while quantifying perox.
Warning(s) suppressed while quantifying ER.
Warning(s) suppressed while quantifying LD.


,image_name,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,surface_area,volume,SA_to_volume_ratio,equivalent_diameter,extent,euler_number,solidity,axis_major_length,min_intensity,max_intensity,mean_intensity,standard_deviation_intensity
0,a24hrs-Ctrl_14_Unmixing,lyso,1,"(0.3891, 0.0799, 0.0799)",0.000000,21.855788,15.930764,0,271,197,1,278,202,0.698442,0.054612,12.789117,0.470721,0.628571,1,inf,0.636832,0.0,6740.0,2750.772727,1982.010364
1,a24hrs-Ctrl_14_Unmixing,lyso,2,"(0.3891, 0.0799, 0.0799)",0.000000,22.349087,19.423787,0,278,241,1,283,246,0.561866,0.039718,14.146382,0.423314,0.640000,1,inf,0.592344,174.0,4247.0,2239.250000,1050.838980
2,a24hrs-Ctrl_14_Unmixing,lyso,3,"(0.3891, 0.0799, 0.0799)",0.000000,22.670667,16.512820,0,279,203,1,289,210,1.025384,0.076954,13.324701,0.527728,0.442857,1,inf,0.792587,315.0,4582.0,2201.580645,1087.198583
3,a24hrs-Ctrl_14_Unmixing,lyso,4,"(0.3891, 0.0799, 0.0799)",0.190035,28.081387,22.904589,0,348,283,2,357,290,1.638994,0.106742,15.354710,0.588544,0.341270,1,0.704918,0.924011,0.0,6208.0,2170.000000,1508.550745
4,a24hrs-Ctrl_14_Unmixing,lyso,5,"(0.3891, 0.0799, 0.0799)",0.440619,28.385563,24.257490,0,352,299,3,360,310,3.146287,0.337603,9.319491,0.863911,0.515152,1,0.839506,1.333907,0.0,8111.0,2645.058824,1895.408771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,a24hrs-Ctrl_14_Unmixing,perox,42,"(0.3891, 0.0799, 0.0799)",5.058540,36.221794,35.702628,13,452,446,14,456,449,0.799834,0.024824,32.220540,0.361929,0.833333,1,inf,0.391290,3727.0,9813.0,7179.700000,2008.598370
143,a24hrs-Ctrl_14_Unmixing,perox,43,"(0.3891, 0.0799, 0.0799)",5.058540,37.891112,34.041298,13,474,425,14,476,428,0.539441,0.012412,43.461666,0.287263,0.833333,1,inf,0.276684,5396.0,14727.0,10237.000000,4098.921663
144,a24hrs-Ctrl_14_Unmixing,perox,44,"(0.3891, 0.0799, 0.0799)",5.447659,24.001431,21.685153,14,300,271,15,302,273,0.444234,0.009929,44.738801,0.266671,1.000000,1,inf,0.178598,7855.0,14096.0,12243.000000,2576.774631
145,a24hrs-Ctrl_14_Unmixing,ER,1,"(0.3891, 0.0799, 0.0799)",2.698696,28.102147,22.608693,0,0,41,16,628,562,2259.962127,229.631999,9.841669,7.597626,0.017670,-11,0.066378,47.058328,0.0,45278.0,8918.054516,4392.976511


### 🧪 **Process <ins>MULTIPLE CELLS</ins> (from a single experiment)**

In [12]:
def _batch_process_org_morph(out_file_name: str,
                            seg_path: Union[Path,str],
                            out_path: Union[Path, str], 
                            raw_path: Union[Path,str], 
                            raw_file_type: str,
                            organelle_names: List[str],
                            organelle_channels: List[int],
                            region_names: List[str],
                            mask: str,
                            scale:bool=True,
                            seg_suffix:Union[str, None]=None) -> int :
    """  
    batch process segmentation quantification (morphology, distribution, contacts); this function is currently optimized to process images from one file folder per image type (e.g., raw, segmentation)
    the output csv files are saved to the indicated out_path folder

    Parameters:
    ----------
    out_file_name: str
        the prefix to use when naming the output datatables
    seg_path: Union[Path,str]
        Path or str to the folder that contains the segmentation tiff files
    out_path: Union[Path, str]
        Path or str to the folder that the output datatables will be saved to
    raw_path: Union[Path,str]
        Path or str to the folder that contains the raw image files
    raw_file_type: str
        the file type of the raw data; ex - ".tiff", ".czi"
    organelle_names: List[str]
        a list of all organelle names that will be analyzed; the names should be the same as the suffix used to name each of the tiff segmentation files
        Note: the intensity measurements collect per region (from get_region_morphology_3D function) will only be from channels associated to these organelles 
    organelle_channels: List[int]
        a list of channel indices associated to respective organelle staining in the raw image; the indices should listed in same order in which the respective segmentation name is listed in organelle_names
    region_names: List[str]
        a list of regions, or masks, to measure; the order should correlate to the order of the channels in the "masks" output segmentation file
    masks_file_name: str
        the suffix of the "masks" segmentation file; ex- "masks_B", "masks", etc.
        this function currently does not accept indivial region segmentations 
    mask: str
        the name of the region to use as the mask when measuring the organelles; this should be one of the names listed in regions list; usually this will be the "cell" mask
    dist_centering_obj:str
        the name of the region or object to use as the centering object in the get_XY_distribution function
    dist_num_bins: int
        the number of bins for the get_XY_distribution function
    dist_center_on: bool=False,
        for get_XY_distribution:
        True = distribute the bins from the center of the centering object
        False = distribute the bins from the edge of the centering object
    dist_keep_center_as_bin: bool=True
        for get_XY_distribution:
        True = include the centering object area when creating the bins
        False = do not include the centering object area when creating the bins
    dist_zernike_degrees: Union[int, None]=None
        for get_XY_distribution:
        the number of zernike degrees to include for the zernike shape descriptors; if None, the zernike measurements will not 
        be included in the output
    include_contact_dist:bool=True
        whether to include the distribution of contact sites in get_contact_metrics_3d(); True = include contact distribution
    scale:bool=True
        a tuple that contains the real world dimensions for each dimension in the image (Z, Y, X)
    seg_suffix:Union[str, None]=None
        any additional text that is included in the segmentation tiff files between the file stem and the segmentation suffix
    


    Returns:
    ----------
    count: int
        the number of images processed
        
    """
    start = time.time()
    count = 0

    # create path objects from strings
    if isinstance(raw_path, str): raw_path = Path(raw_path)
    if isinstance(seg_path, str): seg_path = Path(seg_path)
    if isinstance(out_path, str): out_path = Path(out_path)
    
    # create directory is it doesn't exist
    if not Path.exists(out_path):
        Path.mkdir(out_path)
        print(f"making {out_path}")
    

    # reading list of files from the raw path
    img_file_list = list_image_files(raw_path, raw_file_type)
    len_file_list = len(img_file_list)

    # list of organelle segmentation and masks files to collect from each image
    segs_to_collect = organelle_names + region_names

    # containers to collect data tabels
    org_tabs = []

    # loop through list of cell analyzing each and appending the data to the empty list
    for img_f in img_file_list:
        img_start = time.time()
        count = count + 1
        filez = find_segmentation_tiff_files(img_f, segs_to_collect, seg_path, seg_suffix)

        # read in raw file and metadata
        img_data, meta_dict = read_czi_image(filez["raw"])

        # create intensities from raw file as list based on the channel order provided
        intensities = [img_data[ch] for ch in organelle_channels]

        # store organelle images as list
        organelles = [read_tiff_image(filez[org]) for org in organelle_names]

        # load regions as a list based on order in list (should match order in "masks" file)
        regions = [read_tiff_image(filez[r]) for r in region_names] 

        # define the scale
        if scale is True:
            scale_tup = meta_dict['scale']
        else:
            scale_tup = None

        org_metrics = _get_org_morphology(source_file=img_f,
                                            list_obj_names=organelle_names,
                                            list_obj_segs=organelles,
                                            list_intensity_img=intensities, 
                                            list_region_names=region_names,
                                            list_region_segs=regions, 
                                            mask=mask,
                                            scale=scale_tup)

        org_tabs.append(org_metrics)
        end2 = time.time()
        print(f"Completed quantification of {meta_dict['file_name']} in {(end2-img_start)/60} mins.")
        print(f"{count}/{len_file_list} images have been processed.")
        print(f"Time elapsed: {(end2-start)/60} mins")

    final_org = pd.concat(org_tabs, ignore_index=True)

    org_csv_path = out_path / f"{out_file_name}_organelles.csv"
    final_org.to_csv(org_csv_path)

    end = time.time()
    print(f"Quantification for {count} files is COMPLETE! Files saved to '{out_path}'.")
    print(f"It took {(end - start)/60} minutes to quantify these files.")
    return final_org

In [13]:
batch_org_morph_table = _batch_process_org_morph(out_file_name = "20241204_test_",
                                                seg_path = seg_data_path,
                                                out_path = quant_data_path, 
                                                raw_path = raw_data_path, 
                                                raw_file_type = raw_img_type,
                                                organelle_names = org_file_names,
                                                organelle_channels = org_channels_ordered,
                                                region_names = regions_file_names,
                                                mask = mask_name,
                                                scale = True,
                                                seg_suffix = suffix_separator)

batch_org_morph_table

Quantifying organelle morphology from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw_two\a24hrs-Ctrl_14_Unmixing.czi.
Warning(s) suppressed while quantifying lyso.
Warning(s) suppressed while quantifying mito.
Warning(s) suppressed while quantifying golgi.
Warning(s) suppressed while quantifying perox.
Warning(s) suppressed while quantifying ER.
Warning(s) suppressed while quantifying LD.
Completed quantification of C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw_two\a24hrs-Ctrl_14_Unmixing.czi in 0.5446920394897461 mins.
1/2 images have been processed.
Time elapsed: 0.5447086572647095 mins
Quantifying organelle morphology from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw_two\a48hrs-Ctrl + oleic acid_01_Unmixing.czi.
Warning(s) suppressed while quantifying lyso.
Warning(s) suppressed while quantifying mito.
Warning(s) suppressed while quantifying golgi.
Warning(s) suppressed while quantifying perox.
Warning(s) suppressed while quantifying ER.
Warning(s) s

,image_name,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,surface_area,volume,SA_to_volume_ratio,equivalent_diameter,extent,euler_number,solidity,axis_major_length,min_intensity,max_intensity,mean_intensity,standard_deviation_intensity
0,a24hrs-Ctrl_14_Unmixing,lyso,1,"(0.3891, 0.0799, 0.0799)",0.000000,21.855788,15.930764,0,271,197,1,278,202,0.698442,0.054612,12.789117,0.470721,0.628571,1,inf,0.636832,0.0,6740.0,2750.772727,1982.010364
1,a24hrs-Ctrl_14_Unmixing,lyso,2,"(0.3891, 0.0799, 0.0799)",0.000000,22.349087,19.423787,0,278,241,1,283,246,0.561866,0.039718,14.146382,0.423314,0.640000,1,inf,0.592344,174.0,4247.0,2239.250000,1050.838980
2,a24hrs-Ctrl_14_Unmixing,lyso,3,"(0.3891, 0.0799, 0.0799)",0.000000,22.670667,16.512820,0,279,203,1,289,210,1.025384,0.076954,13.324701,0.527728,0.442857,1,inf,0.792587,315.0,4582.0,2201.580645,1087.198583
3,a24hrs-Ctrl_14_Unmixing,lyso,4,"(0.3891, 0.0799, 0.0799)",0.190035,28.081387,22.904589,0,348,283,2,357,290,1.638994,0.106742,15.354710,0.588544,0.341270,1,0.704918,0.924011,0.0,6208.0,2170.000000,1508.550745
4,a24hrs-Ctrl_14_Unmixing,lyso,5,"(0.3891, 0.0799, 0.0799)",0.440619,28.385563,24.257490,0,352,299,3,360,310,3.146287,0.337603,9.319491,0.863911,0.515152,1,0.839506,1.333907,0.0,8111.0,2645.058824,1895.408771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,a48hrs-Ctrl + oleic acid_01_Unmixing,LD,15,"(0.3891, 0.0799, 0.0799)",2.334711,45.420346,29.521450,6,567,367,7,572,373,1.263401,0.044683,28.274919,0.440265,0.600000,1,inf,0.655341,3132.0,8665.0,6092.611111,1622.816795
325,a48hrs-Ctrl + oleic acid_01_Unmixing,LD,22,"(0.3891, 0.0799, 0.0799)",3.502066,37.290077,19.463723,9,465,242,10,470,247,1.092028,0.039718,27.494555,0.423314,0.640000,1,inf,0.420816,4455.0,9287.0,6694.187500,1300.366449
326,a48hrs-Ctrl + oleic acid_01_Unmixing,LD,30,"(0.3891, 0.0799, 0.0799)",4.669422,11.821004,33.952397,12,145,422,13,152,430,2.045553,0.114189,17.913714,0.601925,0.821429,1,inf,0.698000,4160.0,9825.0,7358.869565,1327.667874
327,a48hrs-Ctrl + oleic acid_01_Unmixing,LD,31,"(0.3891, 0.0799, 0.0799)",4.669422,15.008609,38.890233,12,186,485,13,191,490,1.263692,0.054612,23.139358,0.470721,0.880000,1,inf,0.491397,3791.0,9553.0,6445.000000,1654.984427


### 🧮 **Summarize Metrics per Cell Across One or More Experiments**

-----

## **QUANTIFICATION**